# Ensemble Level LSTM Model

In [2]:
from __future__ import print_function
import os
import time
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import os
import datetime
import pickle
from sklearn import datasets, metrics, cross_validation
from urllib import urlretrieve
import matplotlib.pyplot as plt
import random

URL = "https://github.com/anucc/metatone-analysis/raw/master/metadata/"
PICKLE_FILE = "metatone_performances_dataframe.pickle"

if not os.path.exists(PICKLE_FILE):
    urlretrieve(URL + PICKLE_FILE, PICKLE_FILE)

with open(PICKLE_FILE, 'rb') as f:
        metatone_dataset = pickle.load(f)
        
## Int values for Gesture codes.
NUMBER_GESTURES = 9
GESTURE_CODES = {
    'N': 0,
    'FT': 1,
    'ST': 2,
    'FS': 3,
    'FSA': 4,
    'VSS': 5,
    'BS': 6,
    'SS': 7,
    'C': 8}

vocabulary_size = len(GESTURE_CODES)

In [43]:
## Isolate the Interesting Performances
improvisations = metatone_dataset[
    (metatone_dataset["performance_type"] == "improvisation") &
    (metatone_dataset["performance_context"] != "demonstration") &
    (metatone_dataset["number_performers"] == 4)]
gesture_data = improvisations['gestures']
#metatone_dataset["number_performers"]
ensemble_improvisations = gesture_data.tolist()

## Setup the epochs
## Each batch is of single gestures as input and tuples of remaining performers as output
def generate_epochs(num_epochs, num_steps, batch_size):
    ## Setup the inputs and label sets
    imp_xs = []
    imp_ys = []
    
    for imp in ensemble_improvisations:
        for i in range(len(imp)-num_steps):
            imp_slice = imp[i:i+num_steps]
            for j in range(len(imp_slice.T)):
                imp_x = imp_slice.T[j] # just one input gesture
                imp_y = imp_slice.T[np.arange(len(imp_slice.T)) != j] # indexed by player
                imp_y = imp_y.T # back to indexed by time slice
                imp_xs.append(imp_x)
                imp_ys.append(imp_y)
    
    dataset = zip(imp_xs,imp_ys)
    print("Total Training Examples: " + str(len(imp_xs)))
    print("Total Training Labels: " + str(len(imp_ys)))
    epochs = []
    for j in range(num_epochs):
        # shutffle the big list
        np.random.shuffle(dataset)
        dataset_size = len(dataset)
        batches = []
        for i in range(dataset_size / batch_size):
            ## Setup the batches
            batch = dataset[i*batch_size:(i+1)*batch_size]
            bx,by = zip(*batch)
            batches.append((np.array(bx),np.array(by)))
        epochs.append(batches)
    return epochs


#t = ensemble_improvisations[0]
#len(t[0])
#t.T[np.arange(len(t.T))!=0]
#np.arange(len(t))!=3

# Test 
#a = generate_epochs(2,10,20)
#e1 = a[0]
#e2 = a[1]
#print(len(e1))
#print(len(e2))
#print(e1[0][1].shape)

In [44]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

## Training Network
## Hyperparameters for training
num_nodes = 64
num_classes = vocabulary_size
batch_size = 64
num_steps = 120
num_layers = 3
learning_rate = 1e-4

def load_graph():
    #graph = tf.Graph()
    reset_graph()
    graph = tf.get_default_graph()
    with graph.as_default():
        x = tf.placeholder(tf.int32,[batch_size,num_steps], name='input_placeholder')
        y = tf.placeholder(tf.int32,[batch_size,num_steps], name='labels_placeholder')
        embeddings = tf.get_variable('embedding_matrix', [num_classes, num_nodes])
        rnn_inputs = tf.nn.embedding_lookup(embeddings,x)

        # Define the network
        cell = tf.nn.rnn_cell.LSTMCell(num_nodes,state_is_tuple=True)
        cell = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers, state_is_tuple=True)
        init_state = cell.zero_state(batch_size,tf.float32)
        rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, initial_state=init_state)

        with tf.variable_scope('softmax'):
            W = tf.get_variable('W',[num_nodes,num_classes])
            b = tf.get_variable('b',[num_classes], initializer=tf.constant_initializer(0.0))

        rnn_outputs = tf.reshape(rnn_outputs,[-1,num_nodes])
        y_reshaped = tf.reshape(y,[-1])

        logits = tf.matmul(rnn_outputs, W) + b
        predictions = tf.nn.softmax(logits)

        total_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y_reshaped))
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)
        saver = tf.train.Saver()
        
        
## Training Network
## Hyperparameters for training
num_nodes = 64
num_classes = vocabulary_size
batch_size = 64
num_steps = 120
num_layers = 3
learning_rate = 1e-4

load_graph()
